In [22]:
import numpy as np

In [23]:
file_path = "./ddi_with_type_latest.txt"
with open(file_path, "r") as f:
    readfile = f.read()

In [57]:
reflection_list = []
vector_set = set()
max_vector = 0
for item in readfile.split(sep="\n"):
    if len(item.split(sep="\t")) == 3:
        single_item = item.split(sep="\t")
        reflection_list.append(single_item)
        vector_set.add(single_item[1])
        vector_set.add(single_item[0])
        max_vector = max(max_vector, max(int(single_item[0]), int(single_item[1])))

## Q1 网络中节点和边的数量。

In [69]:
# number of nodes
len(vector_set)

1603

In [70]:
# number of edges
len(reflection_list)

38405

## Q2 找出度最大的前20个节点，并计算所有节点的平均度。

In [76]:
# Use dict to restore the degree of every nodes
node_degree = {str(i):0 for i in range(len(vector_set))}
for reflect in reflection_list:
    node_degree[reflect[0]] += 1
    node_degree[reflect[1]] += 1
node_degree

{'0': 35,
 '1': 43,
 '2': 4,
 '3': 19,
 '4': 1,
 '5': 138,
 '6': 39,
 '7': 2,
 '8': 3,
 '9': 21,
 '10': 1,
 '11': 4,
 '12': 8,
 '13': 21,
 '14': 13,
 '15': 3,
 '16': 12,
 '17': 3,
 '18': 16,
 '19': 85,
 '20': 4,
 '21': 62,
 '22': 162,
 '23': 77,
 '24': 89,
 '25': 64,
 '26': 53,
 '27': 53,
 '28': 97,
 '29': 64,
 '30': 34,
 '31': 52,
 '32': 70,
 '33': 156,
 '34': 54,
 '35': 44,
 '36': 107,
 '37': 133,
 '38': 134,
 '39': 123,
 '40': 77,
 '41': 19,
 '42': 149,
 '43': 60,
 '44': 50,
 '45': 106,
 '46': 85,
 '47': 149,
 '48': 26,
 '49': 61,
 '50': 25,
 '51': 46,
 '52': 59,
 '53': 106,
 '54': 143,
 '55': 50,
 '56': 33,
 '57': 40,
 '58': 22,
 '59': 137,
 '60': 18,
 '61': 45,
 '62': 49,
 '63': 163,
 '64': 34,
 '65': 150,
 '66': 56,
 '67': 76,
 '68': 79,
 '69': 100,
 '70': 37,
 '71': 17,
 '72': 10,
 '73': 1,
 '74': 52,
 '75': 126,
 '76': 20,
 '77': 47,
 '78': 47,
 '79': 2,
 '80': 127,
 '81': 69,
 '82': 37,
 '83': 177,
 '84': 47,
 '85': 100,
 '86': 38,
 '87': 11,
 '88': 174,
 '89': 36,
 '90': 122,

In [77]:
# Order nodes by sum_degree and cut the top20
a = sorted(node_degree.items(), key=lambda x: x[1], reverse=True)
a[:20]

[('848', 194),
 ('350', 188),
 ('870', 186),
 ('118', 181),
 ('83', 177),
 ('972', 176),
 ('88', 174),
 ('1440', 173),
 ('1339', 166),
 ('446', 165),
 ('63', 163),
 ('140', 163),
 ('22', 162),
 ('187', 160),
 ('697', 159),
 ('441', 158),
 ('811', 157),
 ('33', 156),
 ('169', 155),
 ('266', 154)]

In [79]:
# Calculate the average degree of all nodes
average_degree = sum(node_degree.values())/len(vector_set)
average_degree

47.91640673736744

## Q3 统计网络中共有多少种不同类型的边，类型为47的边有多少条

In [86]:
# Count the nums of types
# Weight is equal to the index of type!!!
types = set()
cnt_47 = 0
for reflect in reflection_list:
    types.add(reflect[2])
    if reflect[2] == '47':
        cnt_47 += 1

In [89]:
# Total nums of different edges
len(types)

86

In [90]:
# Nums of edges with weight "47"
cnt_47

6864

## Q4 统计节点度分布,画出度分布图。

## Q5 找出给定节点i和j之间的所有最短路径，并输出。

#### （8,309）、（67,850）、（990,1256）

In [116]:
# djkstra 算法求最短路径
# M 代表两个顶点不可达
M = 1e20

def generate_matrix(node_cnt, array=[]) -> list:
    # Initialize node_cnt * node_cnt Matrix
    matrix = [[M for k in range(int(node_cnt))] for i in range(int(node_cnt))]
    for i in array:
        matrix[i[0]][i[1]] = 1
    return matrix
 
def min_index(arr=[], indexs=[]) -> int:
    # Calculate the node ordinal of the shortest path in a given node
    min_value = arr[0]
    index = indexs[0]
    for i in indexs:
        if arr[i] < min_value:
            min_value = arr[i]
            index = i
    return index
 
def dijkstra(matrix, origin) -> tuple:
    # Use Dijkstra algorithm to find the shortest path in a directed graph
    assert 0 <= origin < len(matrix) 
    s = [origin]
    dist = matrix[origin]
    vs = [i for i in range(len(dist))]
    vs.remove(origin)
    
    p = [-1 if i == M else origin for i in dist]
 
    while len(vs) > 0:
        
        node = min_index(arr=dist, indexs=vs)
        s.append(node)
        vs.remove(node)
        for i in vs:
            if matrix[node][i] != M:
                if dist[node] + matrix[node][i] < dist[i]:
                    dist[i] = dist[node] + matrix[node][i]
                    p[i] = node
    
    dist[origin] = 0
    return dist, p
 
def route(p=[]) -> list:
    i = 0
    result = []
    while i < len(p):
        m = len(p) - 1 - i
        path = [m]
        while p[m] != -1:
            path.append(p[m])
            m = p[m]
        path.reverse()
        result.append((len(p) - 1 - i, path))
        i += 1
    return result
 
if __name__ == '__main__':
    node_cnt = len(vector_set)
    array = []
    for i in reflection_list:
        array.append((int(i[0]), int(i[1])))
    matrix = generate_matrix(node_cnt=node_cnt, array=array)
    start = int(input(u'请输入源位置（0 ~ %d）：' % (int(node_cnt - 1))))
    dist, p = dijkstra(matrix, start)
    target = int(input(u'请输入终点位置（0 ~ %d）：' % (int(node_cnt - 1))))
    for item in route(p):
        if item[0] == target:
            if dist[item[0]] != M:
                print("%d 点到 %d 点的最短距离为 %d，走过的路径为：%s" % (start, item[0], dist[item[0]], item[1]))
            else:
                print("%d 点到 %d 点不可达" % (start, item[0]))

请输入源位置（0 ~ 1602）：8
请输入终点位置（0 ~ 1602）：309
8 点到 309 点不可达
